In [1]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

service_endpoint =   os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name =   'hotelname1'  #os.getenv("AZURE_SEARCH_INDEX_NAME")
key =  os.getenv("AZURE_SEARCH_API_KEY") 


def get_embeddings(text: str):
    # There are a few ways to get embeddings. This is just one example.
    import openai
    open_ai_endpoint = os.getenv("OpenAIEndpoint")
    open_ai_key =  os.getenv("OpenAIKey")

    openai.api_version =   '2023-03-15-preview'#"2022-12-01"
    openai.api_base = open_ai_endpoint
    openai.api_type = "azure"
    openai.api_key = open_ai_key
    model_id =  "opendemo1" # "text-embedding-ada-002"
    embedding = openai.Embedding.create(input=text, deployment_id=model_id)["data"][0]["embedding"]
    return embedding


def get_hotel_index(name: str):
    from azure.search.documents.indexes.models import (
        SearchIndex,
        SearchField,
        SearchFieldDataType,
        SimpleField,
        SearchableField,
        VectorSearch,
        VectorSearchAlgorithmConfiguration,
    )

    fields = [
        SimpleField(name="hotelId", type=SearchFieldDataType.String, key=True),
        SearchableField(name="hotelName", type=SearchFieldDataType.String, sortable=True, filterable=True),
        SearchableField(name="description", type=SearchFieldDataType.String),
        SearchField(
            name="DescriptionVector",
            type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
            searchable=True,
            vector_search_dimensions=1536,
            vector_search_configuration="my-vector-config",
        ),
        SearchableField(
            name="category", type=SearchFieldDataType.String, sortable=True, filterable=True, facetable=True
        ),
    ]
    vector_search = VectorSearch(
        algorithm_configurations=[VectorSearchAlgorithmConfiguration(name="my-vector-config", kind="hnsw")]
    )
    return SearchIndex(name=name, fields=fields, vector_search=vector_search)


def get_hotel_documents():
    docs = [
        {
            "hotelId": "1",
            "hotelName": "Fancy Stay",
            "description": "Best hotel in town if you like luxury hotels.",
            "DescriptionVector": get_embeddings("Best hotel in town if you like luxury hotels."),
            "category": "Luxury",
        },
        {
            "HotelId": "2",
            "HotelName": "Roach Motel",
            "Description": "Cheapest hotel in town. Infact, a motel.",
            "DescriptionVector": get_embeddings("Cheapest hotel in town. Infact, a motel."),
            "Category": "Budget",
        },
        {
            "HotelId": "3",
            "HotelName": "EconoStay",
            "Description": "Very popular hotel in town.",
            "DescriptionVector": get_embeddings("Very popular hotel in town."),
            "Category": "Budget",
        },
        {
            "HotelId": "4",
            "HotelName": "Modern Stay",
            "Description": "Modern architecture, very polite staff and very clean. Also very affordable.",
            "DescriptionVector": get_embeddings(
                "Modern architecture, very polite staff and very clean. Also very affordable."
            ),
            "Category": "Luxury",
        },
        {
            "HotelId": "5",
            "HotelName": "Secret Point",
            "Description": "One of the best hotel in town. The hotel is ideally located on the main commercial artery of the city in the heart of New York.",
            "DescriptionVector": get_embeddings(
                "One of the best hotel in town. The hotel is ideally located on the main commercial artery of the city in the heart of New York."
            ),
            "Category": "Boutique",
        },
    ]
    return docs


def single_vector_search():
    # [START single_vector_search]
    query = "Top hotels in town"

    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

    results = search_client.search(
        search_text="",
        vector=get_embeddings(query),
        top_k=3,
        vector_fields="DescriptionVector",
        select=["hotelId", "hotelName"],
    )

    for result in results:
        print(result)
    # [END single_vector_search]


def single_vector_search_with_filter():
    # [START single_vector_search_with_filter]
    query = "Top hotels in town"

    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

    results = search_client.search(
        search_text="",
        vector=get_embeddings(query),
        top_k=3,
        vector_fields="DescriptionVector",
        filter="category eq 'Luxury'",
        select=["hotelId", "hotelName"],
    )

    for result in results:
        print(result)
    # [END single_vector_search_with_filter]


def simple_hybrid_search():
    # [START simple_hybrid_search]
    query = "Top hotels in town"

    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

    results = search_client.search(
        search_text=query,
        vector=get_embeddings(query),
        top_k=3,
        vector_fields="DescriptionVector",
        select=["hotelId", "hotelName"],
    )
    print(results.get_answers())
    for result in results:
        print(result)
    # [END simple_hybrid_search]


credential = AzureKeyCredential(key)
index_client = SearchIndexClient(service_endpoint, credential)
print(index_name)
index = get_hotel_index(index_name)
print(index)
index_client.create_index(index)
client = SearchClient(service_endpoint, index_name, credential)
print(client)
hotel_docs = get_hotel_documents()
client.upload_documents(documents=hotel_docs)

hotelname1
{'additional_properties': {}, 'name': 'hotelname1', 'fields': [<azure.search.documents.indexes.models._index.SearchField object at 0x000001D3D62E3DF0>, <azure.search.documents.indexes.models._index.SearchField object at 0x000001D3D7B32400>, <azure.search.documents.indexes.models._index.SearchField object at 0x000001D3D7B32370>, <azure.search.documents.indexes.models._index.SearchField object at 0x000001D3D7B323A0>, <azure.search.documents.indexes.models._index.SearchField object at 0x000001D3D7B323D0>], 'vector_search': <azure.search.documents.indexes._generated.models._models_py3.VectorSearch object at 0x000001D3D7B324F0>, 'scoring_profiles': None, 'default_scoring_profile': None, 'cors_options': None, 'suggesters': None, 'analyzers': None, 'tokenizers': None, 'token_filters': None, 'char_filters': None, 'normalizers': None, 'encryption_key': None, 'similarity': None, 'semantic_settings': None, 'e_tag': None}
<SearchClient [endpoint='https://cognitivesearch465.search.window

In [2]:
single_vector_search()

{'hotelName': 'EconoStay', 'hotelId': '3', '@search.score': 0.92625886, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'hotelName': 'Fancy Stay', 'hotelId': '1', '@search.score': 0.9141687, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'hotelName': 'Secret Point', 'hotelId': '5', '@search.score': 0.88500524, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}


In [3]:
single_vector_search_with_filter()

{'hotelName': 'Fancy Stay', 'hotelId': '1', '@search.score': 0.9141687, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'hotelName': 'Modern Stay', 'hotelId': '4', '@search.score': 0.85241705, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}


In [4]:
simple_hybrid_search()

None
{'hotelName': 'EconoStay', 'hotelId': '3', '@search.score': 0.03253968432545662, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'hotelName': 'Fancy Stay', 'hotelId': '1', '@search.score': 0.032522473484277725, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'hotelName': 'Secret Point', 'hotelId': '5', '@search.score': 0.032522473484277725, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
{'hotelName': 'Roach Motel', 'hotelId': '2', '@search.score': 0.01666666753590107, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}
